In [23]:
import dask
import dask.dataframe as dd
from dask.distributed import Client
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import seaborn as sns
from numba import njit
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
import category_encoders as ce

In [23]:
client = Client(n_workers = 16)

tornado.application - ERROR - Multiple exceptions in yield list
Traceback (most recent call last):
  File "/home/cwcollins/anaconda3/lib/python3.7/site-packages/tornado/gen.py", line 883, in callback
    result_list.append(f.result())
  File "/home/cwcollins/anaconda3/lib/python3.7/site-packages/tornado/gen.py", line 1141, in run
    yielded = self.gen.throw(*exc_info)
  File "/home/cwcollins/anaconda3/lib/python3.7/site-packages/distributed/deploy/local.py", line 208, in _start_worker
    yield w._start()
  File "/home/cwcollins/anaconda3/lib/python3.7/site-packages/tornado/gen.py", line 1133, in run
    value = future.result()
  File "/home/cwcollins/anaconda3/lib/python3.7/site-packages/tornado/gen.py", line 1141, in run
    yielded = self.gen.throw(*exc_info)
  File "/home/cwcollins/anaconda3/lib/python3.7/site-packages/distributed/nanny.py", line 157, in _start
    response = yield self.instantiate()
  File "/home/cwcollins/anaconda3/lib/python3.7/site-packages/tornado/gen.py", li

tornado.application - ERROR - Multiple exceptions in yield list
Traceback (most recent call last):
  File "/home/cwcollins/anaconda3/lib/python3.7/site-packages/tornado/gen.py", line 883, in callback
    result_list.append(f.result())
  File "/home/cwcollins/anaconda3/lib/python3.7/site-packages/tornado/gen.py", line 1141, in run
    yielded = self.gen.throw(*exc_info)
  File "/home/cwcollins/anaconda3/lib/python3.7/site-packages/distributed/deploy/local.py", line 208, in _start_worker
    yield w._start()
  File "/home/cwcollins/anaconda3/lib/python3.7/site-packages/tornado/gen.py", line 1133, in run
    value = future.result()
  File "/home/cwcollins/anaconda3/lib/python3.7/site-packages/tornado/gen.py", line 1141, in run
    yielded = self.gen.throw(*exc_info)
  File "/home/cwcollins/anaconda3/lib/python3.7/site-packages/distributed/nanny.py", line 157, in _start
    response = yield self.instantiate()
  File "/home/cwcollins/anaconda3/lib/python3.7/site-packages/tornado/gen.py", li

tornado.application - ERROR - Multiple exceptions in yield list
Traceback (most recent call last):
  File "/home/cwcollins/anaconda3/lib/python3.7/site-packages/tornado/gen.py", line 883, in callback
    result_list.append(f.result())
  File "/home/cwcollins/anaconda3/lib/python3.7/site-packages/tornado/gen.py", line 1141, in run
    yielded = self.gen.throw(*exc_info)
  File "/home/cwcollins/anaconda3/lib/python3.7/site-packages/distributed/deploy/local.py", line 208, in _start_worker
    yield w._start()
  File "/home/cwcollins/anaconda3/lib/python3.7/site-packages/tornado/gen.py", line 1133, in run
    value = future.result()
  File "/home/cwcollins/anaconda3/lib/python3.7/site-packages/tornado/gen.py", line 1141, in run
    yielded = self.gen.throw(*exc_info)
  File "/home/cwcollins/anaconda3/lib/python3.7/site-packages/distributed/nanny.py", line 157, in _start
    response = yield self.instantiate()
  File "/home/cwcollins/anaconda3/lib/python3.7/site-packages/tornado/gen.py", li

tornado.application - ERROR - Multiple exceptions in yield list
Traceback (most recent call last):
  File "/home/cwcollins/anaconda3/lib/python3.7/site-packages/tornado/gen.py", line 883, in callback
    result_list.append(f.result())
  File "/home/cwcollins/anaconda3/lib/python3.7/site-packages/tornado/gen.py", line 1141, in run
    yielded = self.gen.throw(*exc_info)
  File "/home/cwcollins/anaconda3/lib/python3.7/site-packages/distributed/deploy/local.py", line 208, in _start_worker
    yield w._start()
  File "/home/cwcollins/anaconda3/lib/python3.7/site-packages/tornado/gen.py", line 1133, in run
    value = future.result()
  File "/home/cwcollins/anaconda3/lib/python3.7/site-packages/tornado/gen.py", line 1141, in run
    yielded = self.gen.throw(*exc_info)
  File "/home/cwcollins/anaconda3/lib/python3.7/site-packages/distributed/nanny.py", line 157, in _start
    response = yield self.instantiate()
  File "/home/cwcollins/anaconda3/lib/python3.7/site-packages/tornado/gen.py", li

tornado.application - ERROR - Multiple exceptions in yield list
Traceback (most recent call last):
  File "/home/cwcollins/anaconda3/lib/python3.7/site-packages/tornado/gen.py", line 883, in callback
    result_list.append(f.result())
  File "/home/cwcollins/anaconda3/lib/python3.7/site-packages/tornado/gen.py", line 1141, in run
    yielded = self.gen.throw(*exc_info)
  File "/home/cwcollins/anaconda3/lib/python3.7/site-packages/distributed/deploy/local.py", line 208, in _start_worker
    yield w._start()
  File "/home/cwcollins/anaconda3/lib/python3.7/site-packages/tornado/gen.py", line 1133, in run
    value = future.result()
  File "/home/cwcollins/anaconda3/lib/python3.7/site-packages/tornado/gen.py", line 1141, in run
    yielded = self.gen.throw(*exc_info)
  File "/home/cwcollins/anaconda3/lib/python3.7/site-packages/distributed/nanny.py", line 157, in _start
    response = yield self.instantiate()
  File "/home/cwcollins/anaconda3/lib/python3.7/site-packages/tornado/gen.py", li

EOFError: unexpected EOF

In [2]:
%cd ds1-predictive-modeling-challenge

/mnt/e/ds/DS-Unit-3-Sprint-3-Big-Data/module1-aws-sagemaker/ds1-predictive-modeling-challenge


In [3]:
x = dd.read_csv('train_features.csv')
y = dd.read_csv('train_labels.csv')
y = y.drop(["id"],axis=1)

In [4]:
len(x), len(y)

(59400, 59400)

In [5]:
x.columns.values

array(['id', 'amount_tsh', 'date_recorded', 'funder', 'gps_height',
       'installer', 'longitude', 'latitude', 'wpt_name', 'num_private',
       'basin', 'subvillage', 'region', 'region_code', 'district_code',
       'lga', 'ward', 'population', 'public_meeting', 'recorded_by',
       'scheme_management', 'scheme_name', 'permit', 'construction_year',
       'extraction_type', 'extraction_type_group',
       'extraction_type_class', 'management', 'management_group',
       'payment', 'payment_type', 'water_quality', 'quality_group',
       'quantity', 'quantity_group', 'source', 'source_type',
       'source_class', 'waterpoint_type', 'waterpoint_type_group'],
      dtype=object)

In [6]:
x_train, x_test = x.random_split([0.75,0.25],random_state =42)
y_train, y_test = y.random_split([0.75,0.25],random_state =42)

In [7]:
len(x_train),len(y_train)

(44659, 44659)

In [8]:
%%time
Kn = KNeighborsClassifier(n_neighbors=25,weights='distance')
Kn.fit(x_train[["latitude","longitude"]],y_train)
Kn.predict(x_train[["latitude","longitude"]])
Kn.score(x_test[["latitude","longitude"]],y_test)

/home/cwcollins/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


CPU times: user 3.16 s, sys: 609 ms, total: 3.77 s
Wall time: 2 s


In [9]:
z = x[["funder", "installer", "permit", "scheme_management","construction_year","payment","latitude","longitude",]]
z = z.mask(z == 0, np.nan).mask(z == "unknown",np.nan)
z = z.isna()
z["poorly_documented"] = False
for i in z.columns.values:
    z["poorly_documented"] = (z["poorly_documented"] | z[i])
x["poorly_documented"] = z["poorly_documented"]

In [10]:
x["construction_year"] = x["construction_year"].mask(x == 0,2000)

In [11]:
x.columns

Index(['id', 'amount_tsh', 'date_recorded', 'funder', 'gps_height',
       'installer', 'longitude', 'latitude', 'wpt_name', 'num_private',
       'basin', 'subvillage', 'region', 'region_code', 'district_code', 'lga',
       'ward', 'population', 'public_meeting', 'recorded_by',
       'scheme_management', 'scheme_name', 'permit', 'construction_year',
       'extraction_type', 'extraction_type_group', 'extraction_type_class',
       'management', 'management_group', 'payment', 'payment_type',
       'water_quality', 'quality_group', 'quantity', 'quantity_group',
       'source', 'source_type', 'source_class', 'waterpoint_type',
       'waterpoint_type_group', 'poorly_documented'],
      dtype='object')

In [13]:
c = ["basin","region","region_code","district_code","scheme_management","extraction_type","management",\
         "extraction_type_group","quantity_group", "payment", "waterpoint_type","water_quality","quality_group","public_meeting"\
        ,"lga","poorly_documented","source","source_class","0","1","2"]
class_weights = {'functional':1,'non functional':1.2,'functional needs repair':1}

cols1 = ["basin","region","region_code","district_code","scheme_management","extraction_type","management",\
         "extraction_type_group","quantity_group", "payment", "waterpoint_type","water_quality","quality_group",\
         "public_meeting","lga","poorly_documented","source","source_class","extraction_type_class", \
         "top_installer","management_group", "top_funder"]

In [24]:
@njit     

def pipeline():
    pipeline = make_pipeline(
    ce.OneHotEncoder(cols = cols1,use_cat_names=True),
    LogisticRegression(solver='lbfgs',class_weight=class_weights)
    )
    pipeline.fit(x_train[c],y_train)
    y_pred = pipeline.predict(x_test[c])
    accuracy_score(y_test,y_pred)

In [25]:
%%time
pipeline

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 10.3 µs


CPUDispatcher(<function pipeline at 0x7f93419f9a60>)